In [1]:
import pandas as pd
import lightgbm as lgb
import json
from sklearn.model_selection import train_test_split
from collections import Counter

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv('../data/train.csv')
label = pd.read_csv('../data/train_label.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')
train_data = pd.merge(train,label,on='ID')



/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_data.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数,Label
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,...,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0,0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10,0.0,42:05.0,9.0,0.0,266000,NaN,19.160223,00:00.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,00:00.0,5439.0,...,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,319.205293,0.0,0


In [4]:
def deal_time(x):
    if str(x) == 'nan':
        return x
    
    x = str(x)
    parts = x.split(':')
    return int(parts[0]) + float(parts[1])/100

In [5]:
def cal_diff(x, y):
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    else:
        return y-x

In [6]:
def deal_manage_range(x):
    g = json.loads(x)
    return len(g)

In [7]:
# import json
# ct = []
# jyfw = list(train_data['经营范围'])
# for i in jyfw:
#     g = json.loads(i)
#     ct+=g
# dic = dict(Counter(ct))
# len(dic)
# def deal_manage_range_bin(x):
    

In [8]:
all_ranges = []
jyfw = list(train_data['经营范围'])
for item in jyfw:
    all_ranges += json.loads(item)
range_dict = dict(Counter(all_ranges))
# bins = [4,8,16,64,128,256,600,3000]
def deal_manage_range_bin(x, lower, upper=100000):
    count = 0
    items = json.loads(x)
    for item in items:
        counts = range_dict.get(item)
        if counts<=upper and counts>lower:
            count += 1
    return count

In [9]:
train_data['经营期限至'] = train_data['经营期限至'].apply(deal_time)
train_data['经营期限自'] = train_data['经营期限自'].apply(deal_time)
train_data['成立日期'] = train_data['成立日期'].apply(deal_time)
train_data['核准日期'] = train_data['核准日期'].apply(deal_time)
train_data['注销时间'] = train_data['注销时间'].apply(deal_time)
train_data['经营范围1'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
train_data['经营范围2'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
train_data['经营范围3'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(8,16,))
train_data['经营范围4'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(16,64,))
train_data['经营范围5'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(64,128,))
train_data['经营范围6'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(128,256,))
train_data['经营范围7'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(256,600,))
train_data['经营范围8'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(600,3000,))
train_data['经营范围9'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(3000,))
train_data['经营范围'] = train_data['经营范围'].apply(deal_manage_range)

for item in train_data.columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            train_data[parts[0] + '_diff'] = train_data.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

In [10]:
train_data.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,应收账款_diff,一年内到期的非流动资产_diff,油气资产_diff,应收补贴款_diff,应收股利_diff,应收利息_diff,应收票据_diff,预付款项_diff,资产总计_diff,在建工程_diff
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,0.00,5154.0,...,-73.061722,0.0,0.0,0.0,0.0,0.0,25.475114,-3.187707,-212.130364,0.0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,0.00,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13.31,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0.0,42.05,9.0,0.0,266000,NaN,19.160223,0.00,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,0.00,5439.0,...,50.787508,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,-78.322320,0.0


In [11]:
drop_feat = ['ID','邮政编码','Label']
feat = [item for item in train_data.columns if item not in drop_feat]

X = train_data[feat]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [12]:
params = {'num_leaves': 60, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
          'min_data_in_leaf': 30,
          'objective': 'binary', #定义的目标函数
          'max_depth': -1,
          'learning_rate': 0.03,
          "min_sum_hessian_in_leaf": 6,
          "is_unbalance": True,
          "boosting": "gbdt",
          "feature_fraction": 0.9,  #提取的特征比率
          "bagging_freq": 1,
          "bagging_fraction": 0.8,
          "bagging_seed": 11,
          "lambda_l1": 0.1,             #l1正则
          # 'lambda_l2': 0.001,     #l2正则
          "verbosity": -1,
          "nthread": -1,                #线程数量，-1表示全部线程，线程越多，运行的速度越快
          'metric': {'binary_logloss', 'auc'},  ##评价函数选择
          "random_state": 2019, #随机数种子，可以防止每次运行的结果不一致
          # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
          }



In [13]:
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_test, label=y_test)

In [14]:
num_round = 1000
clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=20,
                    early_stopping_rounds=60)

Training until validation scores don't improve for 60 rounds.
[20]	training's binary_logloss: 0.162372	training's auc: 0.969463	valid_1's binary_logloss: 0.172317	valid_1's auc: 0.918085
[40]	training's binary_logloss: 0.161695	training's auc: 0.972217	valid_1's binary_logloss: 0.186243	valid_1's auc: 0.919585
[60]	training's binary_logloss: 0.169029	training's auc: 0.974741	valid_1's binary_logloss: 0.205258	valid_1's auc: 0.919853
Early stopping, best iteration is:
[16]	training's binary_logloss: 0.165653	training's auc: 0.968196	valid_1's binary_logloss: 0.171731	valid_1's auc: 0.917827


In [19]:
pd.set_option('max_rows', None)
feature_importance_df = pd.DataFrame()
feature_importance_df["Feature"] = feat
feature_importance_df["importance"] = clf.feature_importance()
feature_importance_df.sort_values('importance')

,Feature,importance
261,在建工程_diff,0
119,生产性生物资产_年初数,0
120,商誉_年初数,0
121,投资性房地产_年初数,0
123,无形资产及其他资产合计_年初数,0
212,预计负债_diff,0
125,一年内到期的非流动资产_年初数,0
126,油气资产_年初数,0
127,应收补贴款_年初数,0
128,应收股利_年初数,0


In [16]:
prob_oof= clf.predict(x_test, num_iteration=clf.best_iteration)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, prob_oof)
score

0.917826938420251

In [ ]:
0.92282:0.9195909189816495

In [16]:
test['经营期限至'] = test['经营期限至'].apply(deal_time)
test['经营期限自'] = test['经营期限自'].apply(deal_time)
test['成立日期'] = test['成立日期'].apply(deal_time)
test['核准日期'] = test['核准日期'].apply(deal_time)
test['注销时间'] = test['注销时间'].apply(deal_time)
test['经营范围'] = test['经营范围'].apply(deal_manage_range)

for item in test.columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            test[parts[0] + '_diff'] = test.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

In [19]:
test_X = test[feat]
result = clf.predict(test_X)

In [20]:
submission = pd.DataFrame()
submission['ID'] = test['ID']
submission['Label'] = result
submission

,ID,Label
0,0,0.041123
1,2,0.093106
2,4,0.117994
3,5,0.057488
4,6,0.062797
5,7,0.047340
6,8,0.048198
7,11,0.039581
8,12,0.091229
9,13,0.043343


In [22]:
submission.to_csv('submission.csv', index=False, encoding='UTF8')